## Load Required Libraries
<br> 
**Standard Process** <br>
 - ```pip install pandas```<br>
 - ```pip install sqlalchemy```<br> 
 
**Parallel Process** (Not Recommended) <br>
 - ```pip install dask```<br>
 - ```pip install toolz```<br>
 
**SQL Server Driver** <br>
 - ```pip install pymssql```
 - or ```pip install pyodbc```<br>
 
**MySQL Driver** <br>
 - ```pip install pymysql```<br>
 
**Google's BigQuery Driver** <br>
 - ```pip install google-cloud-bigquery```<br>
 - ```pip install google-cloud-bigquery-storage```<br>
 - ```pip install pybigquery```<br>
 - ```pip install pandas-gbq```<br>
 
**PostGreSQL Driver** <br>
 - ```pip install psycopg2```<br>
 
**SQLite Driver** <br>
 - I'm already forgot =_="

In [1]:
## Load Password from my PC
import os, re,json
import pandas as pd

with open('setting.txt','r') as f:
    setting = json.load(f)

# Working with da_tran_SQL 

In [2]:
from py_topping.data_connection.database import da_tran_SQL

## SQL Server (or MS SQL)

In [8]:
set_mssql = setting['MSSQL']
print(set_mssql['type'])

MSSQL


### Create connection to server

In [9]:
mssql = da_tran_SQL(sql_type = 'MSSQL', #set_mssql['type'], #
                host_name = '{YOUR HOST NAME or ID Address}', #set_mssql['host'], #
                database_name = '{Your Database Name}', #set_mssql['database'], #
                user = '{Your Username}', #set_mssql['user'], #
                password = '{Your Password}') #set_mssql['password']) #

Connection OK


for SQL Server the <b>sql_type must == "MSSQL" (ignore case sensitive in version >= 0.1.3)</b>
  
    
by Default, the class will using pymssql as driver and port 1433
    
But you could also change them if you want like below

In [6]:
mssql = da_tran_SQL(sql_type = 'MSSQL', #set_mssql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_mssql['host'],
                database_name = '{Your Database Name}', #set_mssql['database'],
                user = '{Your Username}', #set_mssql['user'],
                password = '{Your Password}',#set_mssql['password'])
                port = '1433',
                driver = 'pymssql')

Connection OK


**from version >= 0.2.2**<br>
You can add more parameter in class by passing to parameter

In [8]:
mssql = da_tran_SQL(sql_type = 'MSSQL', #set_mssql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_mssql['host'],
                database_name = '{Your Database Name}', #set_mssql['database'],
                user = '{Your Username}', #set_mssql['user'],
                password = '{Your Password}',#set_mssql['password'])
                port = '1433',
                driver = 'pyodbc', 
                parameter = 'driver=SQL+Server')

Connection OK


**from version >= 0.1.9**<br>
You could fine tune insert's speed by adjust these parameters into the da_tran_SQL class :
- chunksize, chunksize parameter in pandas' to_sql method (Default Value 150)
- partition_size,  dataframe will divide into smaller size as partition_size before dump (Default Value 5,000)
- parallel_dump, to activate parallel dump (Default Value False) **Risk, not recommend to use**
- max_parallel, number of parellel dump execute pertime (Default Value 2)

In [8]:
mssql = da_tran_SQL(sql_type = 'MSSQL', #set_mssql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_mssql['host'],
                database_name = '{Your Database Name}', #set_mssql['database'],
                user = '{Your Username}', #set_mssql['user'],
                password = '{Your Password}',#set_mssql['password'])
                port = '1433',
                driver = 'pymssql',
                chunksize = 150, 
                partition_size = 5000,
                parallel_dump = False,
                max_parallel = 2)

Connection OK


The da_tran_SQL class also has engine method which equal to engine in sqlalchemy

Which I'm going to use it to set up sample environment that include,
 * 2 dataframe
 * table in mssql
 * store procedure in mssql

In [10]:
def setup_sample(class_in):
    test_table = 'unit_test_git'
    df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] 
                        ,'date' : pd.date_range('2020-10-10','2020-10-14') , 'col3' : [1,1,1,1,1]})
    new_df = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] 
                        ,'date' : pd.date_range('2020-10-12','2020-10-16') , 'col3' : [2,2,2,2,2]})
    df1.to_sql(test_table, index = False, if_exists = 'replace', con = mssql.engine)

    sql_q = """ CREATE OR ALTER PROCEDURE unit_test_git_SP (@PARAM1 AS VARCHAR(100))
                AS
                BEGIN
                SELECT * FROM {} WHERE col1 = @PARAM1 
                END""".format(test_table)

    class_in.engine.execute(sql_q)

    return df1, new_df, test_table, 'unit_test_git_SP'

df1, new_df, table_name, sp = setup_sample(mssql)
pd.read_sql('unit_test_git', con = mssql.engine)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


### 1st Method : Read

You could read any table or view from your database by use method "read"

In [7]:
mssql.read(table_name)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


You could also use pass SQL's "WHERE" statement into parameter "condition_in"

to filter the result

In [8]:
mssql.read(table_name, condition_in = 'col1 <= 2')

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1


**from version>= 0.3.5**<br>
You could also use select statement directly with "raw" = True

In [11]:
mssql.read("""select 'foo' as test_col""", raw = True)

,test_col
0,foo


futher more, you could also use store procedure and pass dict parameter with read method

In [9]:
mssql.read(sp, SP = True, param = {'@PARAM1' : 3})

,col1,col2,date,col3
0,3,2,2020-10-12,1


You could select only unique value of selected column

In [10]:
# Coming Soon...

You could group by and use aggregrate function 

In [11]:
# Coming Soon...

### 2nd Method : dump_whole 

You could replace whole table with new dataframe with is method

**Becareful with Columns' Type, I'm suggest to fix columns' type before using this method**

In [18]:
df1, new_df, table_name, sp = setup_sample(mssql)
mssql.read(table_name)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [19]:
mssql.dump_whole(new_df, table_name)

mssql.read(table_name)

Start Filter Existing data from df at  2020-12-21 15:21:48.335225
Dump data to  unit_test_git  End  2020-12-21 15:21:48.796113


,col1,col2,date,col3
0,4,3,2020-10-12,2
1,5,3,2020-10-13,2
2,6,4,2020-10-14,2
3,7,4,2020-10-15,2
4,8,5,2020-10-16,2


By the way, **dump_whole will use columns and types as the New DataFrame**

In [20]:
mssql.dump_whole(new_df.drop('col1', axis = 1) , table_name)

mssql.read(table_name).dtypes

Start Filter Existing data from df at  2020-12-21 15:21:52.871543
Dump data to  unit_test_git  End  2020-12-21 15:21:53.824815


col2             int64
date    datetime64[ns]
col3             int64
dtype: object

In case that you would like to fixe original table's columns and types

You could add parameter **fix_table = True**

In [21]:
df1, new_df, table_name, sp = setup_sample(mssql)

mssql.engine.execute("""ALTER TABLE {} ALTER COLUMN col1 varchar(10)""".format(table_name))

mssql.read(table_name).dtypes

col1            object
col2             int64
date    datetime64[ns]
col3             int64
dtype: object

In [23]:
mssql.dump_whole(new_df.drop('col1', axis = 1), table_name , fix_table = True) 

mssql.read(table_name)

Start Filter Existing data from df at  2020-12-21 15:22:12.178765
Dump data to  unit_test_git  End  2020-12-21 15:22:12.392116


,col1,col2,date,col3
0,None,3,2020-10-12,2
1,None,3,2020-10-13,2
2,None,4,2020-10-14,2
3,None,4,2020-10-15,2
4,None,5,2020-10-16,2


In [24]:
mssql.read(table_name).dtypes

col1            object
col2             int64
date    datetime64[ns]
col3             int64
dtype: object

### 3rd Method : dump_replace

You could update the existing dataframe by using this method

`self.dump_replace(df_in, table_name_in, list_key, math_logic = '', partition_delete = 100000, debug = False)**`

**List of Parameter**
 - **df_in** : DataFrame to dump into Table (Pandas DataFrame)
 - **table_name_in** : Name of Table (Str)
 - **list_key** : List of key columns (Str or List)
 - **math_logic** : Use math logic to filter your table (Dict), default : ''
 - **partition_delete** : Number of row delete per time (Int), default : 100,000 *Available 0.3.6*
 - **debug** : To show SQL Query or Not (Boolean), default : False

In [14]:
df1, new_df, table_name, sp = setup_sample(mssql)
mssql.read(table_name)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [15]:
new_df

,col1,col2,date,col3
0,4,3,2020-10-12,2
1,5,3,2020-10-13,2
2,6,4,2020-10-14,2
3,7,4,2020-10-15,2
4,8,5,2020-10-16,2


If col1 is the primary key for table and new_df

we could update the existing key and add new key into table by using dump_replace 

In [16]:
mssql.dump_replace(new_df, table_name, list_key = ['col1'])
mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:16.190836
Delete Last ['col1'] at 2020-12-07 16:32:16.276459
Dump data to  unit_test_git  End  2020-12-07 16:32:16.421067


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


For table with multiple keys, we could just simple add more key into the parameter "list_key"

In [17]:
df1, new_df, table_name, sp = setup_sample(mssql)
mssql.read(table_name)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [18]:
mssql.dump_replace(new_df, table_name, list_key = ['col1','col2'])
mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:17.249637
Delete Last ['col1', 'col2'] at 2020-12-07 16:32:17.331459
Dump data to  unit_test_git  End  2020-12-07 16:32:17.469107


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,4,3,2020-10-12,2
5,5,3,2020-10-13,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


dump_replace also has math logic for some ETL process

In [19]:
df1, new_df, table_name, sp = setup_sample(mssql)

In [20]:
mssql.dump_replace(new_df, table_name, list_key = ['date'], 
                   math_logic = {'date' : {'logic' : '>=', 'value' : new_df['date'].min().date(), 'type' : 'date'}} )

mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:18.201008
Delete Last ['date'] at 2020-12-07 16:32:18.289947
Dump data to  unit_test_git  End  2020-12-07 16:32:18.462276


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,4,3,2020-10-12,2
3,5,3,2020-10-13,2
4,6,4,2020-10-14,2
5,7,4,2020-10-15,2
6,8,5,2020-10-16,2


 math logic also could use with num too

In [21]:
df1, new_df, table_name, sp = setup_sample(mssql)

In [22]:
mssql.dump_replace(new_df, table_name, list_key = ['col1','date'], 
                   math_logic = {'col1' : {'logic' : '>=', 'value' : new_df['col1'].min(), 'type' : 'int'} ,
                                 'date' : {'logic' : '>=', 'value' : new_df['date'].min().date(), 'type' : 'date'}} )

mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:19.256544
Delete Last ['col1', 'date'] at 2020-12-07 16:32:19.336450
Dump data to  unit_test_git  End  2020-12-07 16:32:19.481182


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


You could also combine math_logic and non logic key together

In [23]:
df1, new_df, table_name, sp = setup_sample(mssql)

In [24]:
mssql.dump_replace(new_df, table_name, list_key = ['col1','col2'], 
                   math_logic = {'col1' : {'logic' : '>', 'value' : new_df['col1'].min(), 'type' : 'int'} } )

mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:20.250690
Delete Last ['col1', 'col2'] at 2020-12-07 16:32:20.342339
Dump data to  unit_test_git  End  2020-12-07 16:32:20.489196


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,4,3,2020-10-12,2
5,5,3,2020-10-13,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


Since there're many things going inside this function

This Function also have boolean paramter "debug" to print what's going on inside

In [25]:
mssql.dump_replace(new_df, table_name, list_key = ['col1','date'], 
                   math_logic = {'col1' : {'logic' : '>=', 'value' : new_df['col1'].min(), 'type' : 'int'} ,
                                 'date' : {'logic' : '>=', 'value' : new_df['date'].min().date(), 'type' : 'date'}}
                  ,debug = True)

mssql.read(table_name)

Start delete old data at 2020-12-07 16:32:20.549220
Delete Last ['col1', 'date'] at 2020-12-07 16:32:20.633297
delete from unit_test_git where [col1] >= 4 and [date] >= '2020-10-12'
Dump data to  unit_test_git  End  2020-12-07 16:32:20.767764


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


To be honest, I'd recommend you to use a text and good defined key.

***

### 4th Method : dump_new

dump only non existing key into the table

In [26]:
df1, new_df, table_name, sp = setup_sample(mssql)

mssql.read(table_name)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [27]:
mssql.dump_new(new_df, table_name, list_key = ['col1'])

mssql.read(table_name)

Start Filter Existing data from df at  2020-12-07 16:32:21.531746
Dump data to  unit_test_git  End  2020-12-07 16:32:21.732196


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


Just like dump_replace, you colud also use multiple key at once.

In [28]:
df1, new_df, table_name, sp = setup_sample(mssql)

In [29]:
mssql.dump_new(new_df, table_name, list_key = ['col1','col2'])

mssql.read(table_name)

Start Filter Existing data from df at  2020-12-07 16:32:22.473215
Dump data to  unit_test_git  End  2020-12-07 16:32:22.665463


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1
5,4,3,2020-10-12,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


# MySQL

In [6]:
set_mysql = setting['MYSQL']
print(set_mysql['type'])

MYSQL


### Create connection to server

In [9]:
mysql = da_tran_SQL(sql_type = 'MYSQL', #set_mysql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_mysql['host'],
                database_name = '{Your Database Name}', #set_mysql['database'],
                user = '{Your Username}', #set_mysql['user'],
                password = '{Your Password}')#set_mysql['password'])

Connection OK


Same as SQL Server, For MySQL the <b>sql_type must == "MYSQL" (ignore case sensitive)</b>
<br><br>
by Default, the class will using pymysql as driver and port 3306
<br><br>
But you could also change them if you want like below

In [10]:
mysql = da_tran_SQL(sql_type = 'MYSQL', #set_mysql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_mysql['host'],
                database_name = '{Your Database Name}', #set_mysql['database'],
                user = '{Your Username}', #set_mysql['user'],
                password = '{Your Password}',#set_mysql['password'],
                port = '3306',
                driver = 'pymysql')

Connection OK


<b>Every Method and Function are the same as SQL Server.</b>

# Google's BigQuery

In [3]:
set_bgq = setting['gbq']
print(set_bgq['type'])

bigquery


### Create connection to server

In [5]:
gbq = da_tran_SQL(sql_type = 'bigquery', #set_bgq['type'], #
                host_name = '{YOUR Project ID}', #set_bgq['project_id'], #
                database_name = '{Your Dataset Name}') #set_bgq['dataset']) #

Connection OK


For GCP's BigQuery the <b>sql_type must == "BIGQUERY" (ignore case sensitive)</b>
<br><br>
Follow by Project ID and Dataset's name
<br><br>
Parameter's Names are not related because I never thought this function will be used with GCP's BigQuery in the place.
<br><br>
If you have credential json file, you could add files' path into the "credentials" parameter 

In [6]:
gbq = da_tran_SQL(sql_type = 'bigquery', #set_bgq['type'],
                host_name = '{YOUR Project ID}', #set_bgq['project_id'], #
                database_name = '{Your Dataset Name}', #set_bgq['dataset'], #
                credentials = '{Path to Credentials file (JSON)}') #set_bgq['credentials_path']) #

Connection OK


<b>Every Method and Function are the same as SQL Server.</b>

Because BigQuery not automated data type for us like other SQL<br>
SUGGEST TO **USE KEY COLUMNS TYPE AS GOOD FORMATTING TEXT BEFORE DUMP**

# PostGreSQL

In [13]:
set_plsql = setting['POSTGRESQL']
print(set_plsql['type'])

POSTGRESQL


### Create connection to server

In [14]:
plsql = da_tran_SQL(sql_type = 'POSTGRESQL', #set_plsql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_plsql['host'],
                database_name = '{Your Database Name}', #set_plsql['database'],
                user = '{Your Username}', #set_plsql['user'],
                password = '{Your Password}')#set_plsql['password'])

Connection OK


Same as SQL Server, For PostGreSQL the <b>sql_type must == "POSTGRESQL" (ignore case sensitive)</b>
<br><br>
by Default, the class will using pymysql as driver and port 5432
<br><br>
But you could also change them if you want like below

In [17]:
plsql = da_tran_SQL(sql_type = 'POSTGRESQL', #set_plsql['type'],
                host_name = '{YOUR HOST NAME or ID Address}', #set_plsql['host'],
                database_name = '{Your Database Name}', #set_plsql['database'],
                user = '{Your Username}', #set_plsql['user'],
                password = '{Your Password}', #set_plsql['password'])
                port = '5432',
                driver = 'psycopg2')

Connection OK


<b>Every Method and Function are the same as SQL Server.<br>
Except PostGreSQL still can't use Store Procedure or Function yet</b>

# SQLite

In [18]:
sqlite = da_tran_SQL(sql_type = 'sqlite', 
                    host_name = '{path to database file}', #'test.db', 
                    database_name = '', 
                    user = '', 
                    password = '' )

Connection OK


Same as SQL Server, For SQLite the sql_type must == "SQLITE" (ignore case sensitive)

by Default, the class will using sqlite as driver
 - host_name for SQLite is the path to your .db file
 - database_name, user and password will not be read if you didn't address special driver
 - chunksize for SQLite will reduce to 50 because of SQLite's limit

Every Method and Function are the same as SQL Server.<br>
**Don't forget that SQLite don't have Store Procedure or Datetime Type**

# OracleSQL

In [32]:
# Coming Soon...